In [ ]:
from pennylane import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt
import scipy as sp
from tqdm.notebook import tqdm

In [ ]:
def angstrom_to_bohr(distance):
    return distance / 0.52917721067121
def bohr_to_angstrom(distance):
    return distance * 0.52917721067121

In [ ]:
random_seed = 1111

In [ ]:
def define_problem(num_hydrogens, distance, basis='sto-3g', mapping='jordan_wigner'):
    symbols = ["H"] * num_hydrogens
    coordinates = [[0., 0., 0.]]
    for i in range(1, num_hydrogens):
        coordinates += [[0., 0., angstrom_to_bohr(i * distance)]]
    geometry = np.array(coordinates, requires_grad=True)
    molecule = qml.qchem.Molecule(symbols, geometry)
    H, n_qubits = qml.qchem.molecular_hamiltonian(
        symbols, 
        geometry,
        charge=0,
        mult=1,
        mapping = mapping,
        basis = basis,
        method="pyscf"
    )
    hf = qml.qchem.hf_state(num_hydrogens, n_qubits)
    singles, doubles = qml.qchem.excitations(num_hydrogens, n_qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    return H, n_qubits, s_wires, d_wires

In [ ]:
def plot_energies(exact_data=None, qc_data=None, algorithm_name=None):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)
    if exact_data is not None:
        ax.plot(dist_list_exact, exact_data[0], label = 'Exact Diagonalization', c='k')
        for data in exact_data[1:]:
            ax.plot(dist_list_exact, data, c='k')
    if qc_data is not None:
        ax.scatter(dist_list_quantum, qc_data[0], label = algorithm_name, c='r')
        for data in qc_data[1:]:
            ax.scatter(dist_list_quantum, data, c='r')

    ax.legend()
    ax.set_xlabel('Bond Length (Å)')
    ax.set_ylabel('Energy (Ha)')
#     ax.set_ylim([-1.5, 0.5])
    ax.grid()

In [ ]:
from qiskit.providers.fake_provider import FakeBelem
from qiskit.providers.aer.noise import NoiseModel
device_backend = FakeBelem()

In [ ]:
noise_model_belem = NoiseModel.from_backend(device_backend)

In [ ]:
dev = qml.device("default.qubit", wires=4)

In [ ]:
opt = qml.AdamOptimizer()# stepsize=0.05)

In [ ]:
dist_list_exact = np.linspace(0.25, 2, 51)

In [ ]:
dist_list_quantum = np.linspace(0.25, 2, 11)

## 1. UCCSD Ansatz

In [ ]:
def UCCSD(weights, s_wires, d_wires):
    ## Initialize to HF state
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    for d_weight, d_wire in zip(weights[:len(d_wires)], d_wires):
        qml.FermionicDoubleExcitation(d_weight, wires1=d_wire[0], wires2=d_wire[1])
    for s_weight, s_wire in zip(weights[len(d_wires):], s_wires):
        qml.FermionicSingleExcitation(s_weight, wires=s_wire)

def UCCSD_dagger(weights, s_wires, d_wires):
    ## Initialize to HF state
    for s_weight, s_wire in zip(weights[len(d_wires):][::-1], s_wires[::-1]):
        qml.adjoint(qml.FermionicSingleExcitation(s_weight, wires=s_wire))
        
    for d_weight, d_wire in zip(weights[:len(d_wires)][::-1], d_wires[::-1]):
        qml.adjoint(qml.FermionicDoubleExcitation(d_weight, wires1=d_wire[0], wires2=d_wire[1]))

    qml.PauliX(wires=0)
    qml.PauliX(wires=1)

In [ ]:
def UCCSD_without_initialization(weights, s_wires, d_wires):
    ## Initialize to HF state
#     qml.PauliX(wires=0)
#     qml.PauliX(wires=1)
    for d_weight, d_wire in zip(weights[:len(d_wires)], d_wires):
        qml.FermionicDoubleExcitation(d_weight, wires1=d_wire[0], wires2=d_wire[1])
    for s_weight, s_wire in zip(weights[len(d_wires):], s_wires):
        qml.FermionicSingleExcitation(s_weight, wires=s_wire)


In [ ]:
def hardware_efficient_ansatz(weights, wires):
    qml.templates.StronglyEntanglingLayers(weights=weights, wires=wires)

## 2. Exact Diagonalization

In [ ]:
def run_Exact(H):
    n_qubits = len(H.wires)
    Hmat = qml.utils.sparse_hamiltonian(H).todense()
    eig_vals, eig_vecs = sp.linalg.eigh(Hmat)
    return eig_vals, eig_vecs

In [ ]:
def sweep_Exact(dist_list):
    energies_list = []
    energies_all = []
    for j, dist in enumerate(tqdm(dist_list)):
        H, n_qubits, s_wires, d_wires = define_problem(2, dist)
        eig_vals, eig_vecs = run_Exact(H)
#         energies_list.append(eig_vals)
        energies_valid = []
        for i in range(2**n_qubits):
            if np.abs(
                np.real(eig_vecs[:, i][..., np.newaxis].conjugate().T @ 
                        qml.matrix(qml.qchem.particle_number(4)) @ 
                        eig_vecs[:, i][..., np.newaxis])[0, 0] - 2
            ) < 1e-10:
                energies_valid.append(eig_vals[i])
        energies_list.append(energies_valid)
        energies_all.append(eig_vals)
        
    data = np.array(energies_list)
    data.sort(axis = 1)
    
    data_all = np.array(energies_all)
    data_all.sort(axis = 1)
    return [x for x in data.T], [x for x in data_all.T]

In [ ]:
energies_exact, energies_exact_all = sweep_Exact(dist_list_exact)

In [ ]:
plot_energies(exact_data=energies_exact)

In [ ]:
plot_energies(exact_data=energies_exact_all)

## 2. VQE

In [ ]:
def run_VQE_UCCSD(H, s_wires, d_wires, opt, random_seed=1234, max_iter=1000, r_tol=1e-12):
    n_qubits = len(H.wires)
    np.random.seed(random_seed)
    @qml.qnode(dev)
    def energy_circuit(params):
        UCCSD(params, s_wires, d_wires)
        return qml.expval(H)
    params = np.random.random(len(s_wires) + len(d_wires), requires_grad=True) * 2 * np.pi - np.pi
    energies = [10000. ]
    for i in range(max_iter):
        params, energy = opt.step_and_cost(energy_circuit, params)
        energies.append(energy)
        if (abs((energies[-2] - energies[-1])/energies[-1]) < r_tol):
            break
    return params, energies[1:], energy_circuit

In [ ]:
def sweep_VQE_UCCSD(dist_list, opt, random_seed=1234, max_iter=1000, r_tol=1e-12):
    opt_energy_list = []
    param_list = []
    energies_list = []
    for j, dist in enumerate(tqdm(dist_list)):
        H, n_qubits, s_wires, d_wires = define_problem(2, dist)
        params, energies, energy_circuit = run_VQE_UCCSD(H, s_wires, d_wires, opt, r_tol=r_tol, random_seed=random_seed)
        param_list.append(params)
        energies_list.append(energies)
        opt_energy_list.append(energy_circuit(params))
        
    return [np.array([float(x) for x in opt_energy_list])], param_list, energies_list

In [ ]:
energies_VQE, param_list_VQE, steps_VQE = sweep_VQE_UCCSD(dist_list_quantum, opt, random_seed=random_seed)

In [ ]:
plot_energies(exact_data=energies_exact, qc_data=energies_VQE, algorithm_name='VQE')

## 3. VQD

In [ ]:
def run_VQD_UCCSD(H, s_wires, d_wires, num_excited, opt, random_seed=1234, max_iter=1000, r_tol=1e-12):
    n_qubits = len(H.wires)
    np.random.seed(random_seed)
    eigenstate_params = []
    eigenstate_energies = []
    
    @qml.qnode(dev)
    def energy_circuit(params):
        UCCSD(params, s_wires, d_wires)
        return qml.expval(H)
    
    @qml.qnode(dev)
    def overlap_circuit(params_1, params_2):
        UCCSD(params_1, s_wires, d_wires)    
        UCCSD_dagger(params_2, s_wires, d_wires)    
        return qml.probs(wires=range(4))
    
    beta = 10 * np.sum(np.abs(H.coeffs, requires_grad=True))
    for k_excited in range(num_excited):
        def cost_func(params):
            energy = energy_circuit(params)
            overlap = 0
            for i_eigen, eigenstate_param in enumerate(eigenstate_params):
                tmp_overlap = overlap_circuit(eigenstate_param, params)[0]
                overlap += tmp_overlap
            return energy + beta * overlap
        
        params = np.random.random(len(s_wires) + len(d_wires), requires_grad=True) * 2 * np.pi - np.pi
        energies = [10000. ]
        
        for i in range(max_iter):
            params, energy = opt.step_and_cost(cost_func, params)
            energies.append(energy)
            if (abs((energies[-2] - energies[-1])/energies[-1]) < r_tol):
                break
        eigenstate_params.append(params)
        eigenstate_energies.append(energies[1:])
    return eigenstate_params, eigenstate_energies, energy_circuit

In [ ]:
def sweep_VQD_UCCSD(dist_list, num_excited, opt, random_seed=1234, max_iter=1000, r_tol=1e-12):
    opt_energy_list = []
    param_list = []
    energies_list = []
    for j, dist in enumerate(tqdm(dist_list)):
        H, n_qubits, s_wires, d_wires = define_problem(2, dist)
        params, energies, energy_circuit = run_VQD_UCCSD(
            H, s_wires, d_wires, num_excited, opt, r_tol=r_tol, random_seed=random_seed
        )
        param_list.append(params)
        energies_list.append(energies)
        opt_energy_list.append([energy_circuit(param) for param in params])
        tmp = np.array(opt_energy_list)
        tmp.sort(axis = 1)
    return [x for x in tmp.T], param_list, energies_list

In [ ]:
energies_VQD, param_list_VQD, steps_VQD = sweep_VQD_UCCSD(dist_list_quantum, 4, opt, r_tol=1e-12, random_seed=random_seed)

In [ ]:
np.array(energies_VQD)

In [ ]:
plot_energies(exact_data=energies_exact, qc_data=energies_VQD, algorithm_name='VQD')

## 4. SSVQE

In [ ]:
from itertools import permutations

In [ ]:
def run_SSVQE_UCCSD(H, s_wires, d_wires, num_excited, opt, random_seed=1234, max_iter=1000, r_tol=1e-12):
    n_qubits = len(H.wires)
    np.random.seed(random_seed)
    
    eigenstate_params = []
    eigenstate_energies = []
    
    weights = np.arange(1, num_excited+1)[::-1]
    # Below code for only N = 2 case
    # basises = [''.join([str(y) for y in x]) for x in list(set(permutations([1]*2 + [0]*2, 4)))]
    def comp_basis(index: int):
        for i, x in enumerate(bin(index)[2:].zfill(4)):
            if x == '1':
                qml.PauliX(i)
    
    @qml.qnode(dev)
    def energy_circuit(params, i):
        comp_basis(i)
        UCCSD_without_initialization(params, s_wires, d_wires)     
        return qml.expval(H)

    def cost_func(params):
        cost = 0
        for i in range(num_excited):
            cost += weights[i] * energy_circuit(params, i)
        return cost

    params = np.random.random(len(s_wires) + len(d_wires), requires_grad=True) * 2 * np.pi - np.pi
    costs = [10000. ]

    for i in range(max_iter):
        params, cost = opt.step_and_cost(cost_func, params)
        costs.append(cost)
        step_energies = []
        for j in range(num_excited):
            step_energies.append(energy_circuit(params, j))
        eigenstate_energies.append(step_energies)
        eigenstate_params.append(params)
        if (abs((costs[-2] - costs[-1])/costs[-1]) < r_tol):
            break
    return eigenstate_params, eigenstate_energies, energy_circuit

In [ ]:
def sweep_SSVQE_UCCSD(dist_list, num_excited, opt, random_seed=1234, max_iter=10000, r_tol=1e-6):
    param_list = []
    energies_list = []
    for j, dist in enumerate(tqdm(dist_list)):
        H, n_qubits, s_wires, d_wires = define_problem(2, dist)
        params, energies, energy_circuit = run_SSVQE_UCCSD(
            H, s_wires, d_wires, num_excited, opt, r_tol=r_tol, random_seed=random_seed
        )
        param_list.append(params)
        energies_list.append(energies)
        tmp = np.array([x[-1] for x in energies_list]).T
        tmp.sort(axis = 0)
    return [x for x in tmp], param_list, energies_list

In [ ]:
energies_SSVQE, param_list_SSVQE, steps_SSVQE = sweep_SSVQE_UCCSD(dist_list_quantum, 16, opt, r_tol=1e-12, random_seed=random_seed)

In [ ]:
plot_energies(exact_data=energies_exact_all, qc_data=energies_SSVQE, algorithm_name='SSVQE')

In [ ]:
plot_energies(exact_data=energies_exact, qc_data=energies_SSVQE, algorithm_name='SSVQE')